# Single Qubit Rotations in Measurement Based Quantum Computation

In this session you will show how measurements, or more specifically the choice of measurement basis, can be used to perform arbitrary single-qubit rotations. This is one of the ingredients of measurement based quantum computation that you have learned about in the lecture. We will also program a measurement based rotation protocol for a gate model quantum computer!

Let us start with the following *half-teleportation circuit*:

<div><img src="pics/half_tel.jpg" width="310"></div>

Here, the meter symbol indicates a measurement of the observable

$$
	O(\varphi) = \cos \varphi X + \sin \varphi Y \,,
$$

i.e. measurement along a basis in the $x$-$y$-plane with angle $\varphi$. The measurement basis is graphically shown on the Bloch sphere by the pink arrow below. Note that for readability, the Bloch sphere is rotated from it's 'regular' orientation such that $|+\rangle$ is on the right side of the sphere, rather than the 'front'.

<div><img src="pics/basis.svg" width="250"></div>

> ***Exercise 1.*** Verify that the eigenvectors of $O(\varphi)$ are 
> 	
> $$\left|\varphi \pm \right> = \left(e^{-i\varphi/2}\left|0\right> \pm e^{i\varphi/2}
	\left|1\right> \right)/\sqrt{2}$$ 
>
> with eigenvalues $\pm 1$. These are the two states that our measurement can distinguish.

> ***Exercise 2.*** Given an arbitrary input state $\left|\psi_{in}\right> = \alpha \left|0\right> + \beta
	\left|1\right>$, show that $\left|\psi_{out}\right> = \underbrace{ H e^{i\varphi
	Z/2}Z^{s}}_{U}\left|\psi_{in}\right>$ where $s$ defines the measured eigenvalue $(-1)^{s}$ of the operator $O(\varphi)$.

We can now take four half-teleportation circuits and add them in a chain, feeding each output as input into the next circuit. The result looks like this:

<div><img src="pics/mbqc_rot.jpg" width="500"></div>

At each stage we perform a measurement in the $x$-$y$-plane, but we allow for differing angles $\varphi_i$. You have shown in exercise 2 that a single stage acts as a unitary

$$
U_i = H e^{i\varphi_i	Z/2}Z^{s_i}
$$

on the input state, $\left|\psi_{out}\right> =U_i \left|\psi_{in}\right>$. Thus, a chain with four stages yields the following unitary


$$
	\left|\psi_{out}\right> = 
    U_4 U_3 U_2 U_1 \left|\psi_{out}\right> =
	\left( He^{-i\varphi_4 Z/2} Z^{s_4} \right)
	\left( He^{-i\varphi_3 Z/2} Z^{s_3} \right)
	\left( He^{-i\varphi_2 Z/2} Z^{s_2} \right)
	\left( He^{-i\varphi_1 Z/2} Z^{s_1} \right)
	\left|\psi_{in}\right>
$$



After a bit of algebra and setting $\varphi_1=0$, the unitary simplifies to (up to a global phase)

$$
Z^{s_1+s_3} X^{s_2+s_4} \cdot 
\exp \left\{i(-1)^{s_1+s_3} \varphi_4 \frac{X}{2} \right\}
\exp \left\{i(-1)^{s_2} \varphi_3 \frac{Z}{2} \right\}
\exp \left\{i(-1)^{s_1} \varphi_2 \frac{X}{2} \right\} \,.
$$


> ***Take-home exercise.*** Perform this simplification yourself. You might find the relation $HZ=XH$ helpful. Also remember the Pauli matrix algebra.

Finally we set 
$$
\begin{aligned}
    \varphi_1 &= 0 \\
	\varphi_2 &= -(-1)^{s_1} \xi \\
	\varphi_3 &= -(-1)^{s_2} \eta \\
	\varphi_4 &= -(-1)^{s_1+s_3} \zeta
\end{aligned}
$$

to get

$$
\left|\psi_{out}\right> = \underbrace{Z^{s_1+s_3} X^{s_2+s_4} }_{U_P}
\exp\left\{-i\frac{\zeta}{2}\sigma_x\right\}
\exp\left\{-i\frac{\eta}{2}\sigma_z\right\}
\exp\left\{-i\frac{\xi}{2}\sigma_x\right\}
\left|\psi_{in}\right> \,.
$$

But the second part is a general qubit rotation with [Euler angles](https://mathworld.wolfram.com/EulerAngles.html) $\xi,\eta,\zeta$! The first bit, $U_P$, is a Pauli correction that comes as an unwanted byproduct. We can remove these Paulis by postprocessing the data.


> ***Exercise 3.*** Usually, the order in which the qubits of a quantum circuit are measured, does not matter. Is that also the case for our quantum protocol? In which order should the qubits be measured?

## Implementing the MBQC Protocol on a Gate-Based Quantum Computer

For MBQC to be useful, we don't want to build cluster states on a gate-based quantum computer. First of all, our circuit above uses five *physical* qubits to implement the rotation of a single *logical* qubit. This is a big qubit overhead. Secondly, measurements on a gate-based quantum computer in the $x$-$y$ plane require the ability to do single qubit rotations, which is the very thing that we want to introduce using the MBQC protocol.

The deep idea is to look for some quantum mechanical system that naturally implements most of the MBQC protocol. Specifically, we would like to find some system whose groundstate is the *cluster state*, shown below.

<div><img src="pics/cluster_state2.jpg" width="300"></div>

One would then perform measurements on it, choosing the order and measurement angles carefully to effectively realize a meaningful quantum computation. 


That being said, let's do some coding anyways to get a better idea of how MBQC works. The first step is to implement the cluster state.

In [1]:
import pennylane as qml
from pennylane import numpy as np

shots = 80000
dev = qml.device('default.qubit', wires=5, shots=shots)

> ***Exercise 4.*** Complete the function `createClusterState` by coding above quantum circuit. You can choose your favorite input state $\left|\psi_{in}\right>$ for this. The input state corresponds to `wires=0`.

In [2]:
def initializePsi_in():
    qml.Hadamard(wires=0)
    qml.RZ(np.pi/4, wires=0)

def createClusterState():
    initializePsi_in()
    
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.Hadamard(wires=3)
    qml.Hadamard(wires=4)
    
    qml.CZ(wires=[0,1])
    qml.CZ(wires=[1,2])
    qml.CZ(wires=[2,3])
    qml.CZ(wires=[3,4])
   

Next we implement the measurement in the basis specified by the angles $\varphi_1=0,\varphi_2,\varphi_3,\varphi_4$. We can perform such a measurement by adding single qubit gates that rotate the $\varphi_i$-basis into the computational basis.

A possible approach is to rotate the $\varphi$ basis around the $z$ axis into the $x$-basis. Then one can apply a Hadamard gate to rotate $x$ into $z$.


<div><img src="pics/phi_rotation.svg" width="680"></div>


> **Exercise 5.** By completing the function `measurePhiBasis`, perform above described measurement basis rotation on qubits `0-3` using the parameters `phi_2`, `phi_3`, `phi_4`. We have already set $\varphi_1=0$, so that you won't need a variable `phi_1`.


In [3]:
def measurePhiBasis(phi_2, phi_3, phi_4):
    qml.Hadamard(wires=0)
    
    qml.RZ(-phi_2, wires=1)
    qml.Hadamard(wires=1)
    
    qml.RZ(-phi_3, wires=2)
    qml.Hadamard(wires=2)
    
    qml.RZ(-phi_4, wires=3)
    qml.Hadamard(wires=3)

We put all pieces together using the three functions below. To reconstruct the full quantum state, measurement in a single basis is not sufficient, as we have already seen in this workshop. Therefore, we will measure the output qubit in the $x$-, $y$-, and $z$-basis.


In [4]:
@qml.qnode(dev)
def mbqcRotationZ(phi_2, phi_3, phi_4):
    createClusterState()
    measurePhiBasis(phi_2, phi_3, phi_4)
    
    return qml.probs(wires=np.arange(5))

@qml.qnode(dev)
def mbqcRotationX(phi_2, phi_3, phi_4):
    createClusterState()
    measurePhiBasis(phi_2, phi_3, phi_4)
     
    qml.Hadamard(wires=4)
    return qml.probs(wires=np.arange(5))

@qml.qnode(dev)
def mbqcRotationY(phi_2, phi_3, phi_4):
    createClusterState()
    measurePhiBasis(phi_2, phi_3, phi_4)
    
    qml.S(wires=4).inv()
    qml.Hadamard(wires=4)
    
    return qml.probs(wires=np.arange(5))

Finally, we have to discuss the data processing strategy, i.e. how to deal with the byproduct operator $U_P$. A direct application of the Pauli gates on the quantum computer is not possible, since measurements currently cannot depend on outcomes of other measurements. So instead we classically applied the Pauli correction $U_P$ ourselves by post-processing the data.

However, now we have an additional problem. The *measurement angles* depend on the outcomes $s_i$ of other measurements, according to 

$$
\begin{aligned}
    \varphi_1 &= 0 \\
	\varphi_2 &= -(-1)^{s_1} \xi \\
	\varphi_3 &= -(-1)^{s_2} \eta \\
	\varphi_4 &= -(-1)^{s_1+s_3} \zeta
\end{aligned}
$$

This cannot be easily fixed with a Pauli matrix. Instead, our strategy will be to try all possible combinations of measurements angles, i.e. by running 8 circuits with all possible choices of $s_1,s_2,s_3$. 

Sometimes our choice of $s_1,s_2,s_3$ will match what we actually measure. In that case we will record the measurement. If our guess was wrong, we will simply discard that measurement result. This process is called *postselection*.

Let us sum up all steps as pseudocode:

<ul>
<li>Choose values for $\xi,\eta,\zeta$</li>

<li>Create Input state and cluster state</li>

<li><code>for</code> all possible combinations of` $s_1,s_2,s_3 = \{0,1\}$</li>
   
<ul>
<li>Perform measurement of the first four qubits in the basis specified by

$$
\begin{aligned}
    \varphi_1 &= 0 \qquad
	\varphi_2 = -(-1)^{s_1} \xi \qquad
	\varphi_3 = -(-1)^{s_2} \eta \qquad
	\varphi_4 = -(-1)^{s_1+s_3} \zeta
\end{aligned}
$$
    
</li>

<li>and measure the last qubit in the $x$-, $y$-, and $z$-basis.</li>
        
<li>Check if the actually measured values $\bar{s}_1,\bar{s}_2,\bar{s}_3$ agree with our choice of $s_i$, i.e. check that
$\bar{s}_1=s_1$ and $\bar{s}_2=s_2$ and $\bar{s}_3=s_3$. </li>
    <ul> <li>If no, discard the data.</li>
        <li>If yes, keep the data.</li>
    </ul>
    <li>Apply the Pauli corrections.</li>
</ul>

<li> Compare the data to the expected result. </li>
</ul>

> **Take-home exercise.** The postselection and Pauli correction is implemented in the file `postselection.py`. You don't have to worry about it for now, but you are encouraged to have a look at the code later. You may also write your own implementation of above pseudocode. This likely will be a lot of work, but it will intimately make you familiar with the problem.

In [5]:
# import the file postselection.py
from postselection import *    

Finally, here is the main routine that calls all our functions and records the processed measurement outcomes `counts_x`, `counts_y`, `counts_z` of the output qubit in the $x$-, $y$-, and $z$-basis.

In [12]:
# randomly choose Euler angles that define the single-qubit rotation 
zeta = (np.random.rand()-0.5)*np.pi
eta = (np.random.rand()-0.5)*np.pi
xi = (np.random.rand()-0.5)*np.pi

# these arrays will store measurement results of the output qubit
counts_x = np.zeros(2)
counts_y = np.zeros(2)
counts_z = np.zeros(2)

for b in ['000', '001', '010', '011', '100', '101', '110','111']:
    # loop through all possible combinations of s1,s2,s3
    s1 = int(b[0])
    s2 = int(b[1])
    s3 = int(b[2])
    
    # run the quantum circuit with the measurement angles -(-1)**s1*xi, -(-1)**s2*eta, -(-1)**(s1+s3)*zeta
    # the output qubit is measured in the x-basis
    data = mbqcRotationX(-(-1)**s1*xi, -(-1)**s2*eta, -(-1)**(s1+s3)*zeta) 
    # postselect the data
    counts = postselect(s1, s2, s3, data, 'x', shots)
    # add the correct guesses for specific instance of s1,s2,s3 to the total count
    counts_x += counts
    
    # repeat everything but now output qubit is measured in y-basis
    data = mbqcRotationY(-(-1)**s1*xi, -(-1)**s2*eta, -(-1)**(s1+s3)*zeta) 
    counts = postselect(s1, s2, s3, data, 'y', shots)
    counts_y += counts
    
    # and now in z-basis
    data = mbqcRotationZ(-(-1)**s1*xi, -(-1)**s2*eta, -(-1)**(s1+s3)*zeta) 
    counts = postselect(s1, s2, s3, data, 'z', shots)
    counts_z += counts


How do we verify if our qubit has been rotated successfully? A quick way to check is by directly implementing

$$
\exp\left\{-i\frac{\zeta}{2}\sigma_x\right\}
\exp\left\{-i\frac{\eta}{2}\sigma_z\right\}
\exp\left\{-i\frac{\xi}{2}\sigma_x\right\}
$$

using single qubit gates and no additional measurements. 

> **Exercise 6.** Implement this unitary for a fresh quantum circuit with a single qubit by completing the function below. 

In [13]:
def qubitRotation(xi, eta, zeta):
    initializePsi_in()
    
    qml.RX(xi, wires=0)
    qml.RZ(eta, wires=0)
    qml.RX(zeta, wires=0)
 

The resulting state is again measured in the $x$-, $y$-, and $z$-basis.

In [14]:
@qml.qnode(dev)
def qubitRotationZ(xi, eta, zeta):
    qubitRotation(xi, eta, zeta)
    # return measurement probabilities
    return qml.probs(wires=[0])

@qml.qnode(dev)
def qubitRotationX(xi, eta, zeta):
    qubitRotation(xi, eta, zeta)  
    # to measure in x-basis, use Hadamard gate to map {|+> ,|->} onto {|0>, |1>}
    qml.Hadamard(wires=0)
    return qml.probs(wires=[0])

@qml.qnode(dev)
def qubitRotationY(xi, eta, zeta):
    qubitRotation(xi, eta, zeta)
    # to measure in y-basis, use (H S^dagger) to map {|y+> ,|y->} onto {|0>, |1>}
    qml.S(wires=0).inv()
    qml.Hadamard(wires=0)
    return qml.probs(wires=[0])

Let's compare the two datasets! If you implemented everything correctly, they should agree!

In [15]:
print('The MBQC protocoll gave the following probabilities for [|+>,|->]:',counts_x/shots)
print('With a direct single qubit gate implementation we get:', qubitRotationX(xi, eta, zeta))

The MBQC protocoll gave the following probabilities for [|+>,|->]: [0.4303375 0.5711   ]
With a direct single qubit gate implementation we get: [0.433 0.567]


In [16]:
print('The MBQC protocoll gave the following probabilities for [|y+>,|y->]:',counts_y/shots)
print('With a direct single qubit gate implementation we get:', qubitRotationY(xi, eta, zeta))

The MBQC protocoll gave the following probabilities for [|y+>,|y->]: [0.7061875 0.2929625]
With a direct single qubit gate implementation we get: [0.7047875 0.2952125]


In [17]:
print('The MBQC protocoll gave the following probabilities for [|0>,|1>]:',counts_z/shots)
print('With a direct single qubit gate implementation we get:', qubitRotationZ(xi, eta, zeta))

The MBQC protocoll gave the following probabilities for [|0>,|1>]: [0.0493625 0.950325 ]
With a direct single qubit gate implementation we get: [0.0501 0.9499]


> ***Exercise 7.*** You have chosen input state $\left|\psi_{in}\right>$ in your definition of the function `initializePsi_in`. Test at least one more input state and verify that the MBQC protocol still works as expected.